# Run FunSearch on Bin Packing
Five steps:
1. Implement 'LLM' interface.
2. Implement a 'SandBox' interface.
3. Prepare a 'specification'.
4. Prepare a dataset.
5. Start FunSearch.

## Preparation: download the project file from github. And update system path.

In [ ]:
"""import shutil

folder_path = "/content/TSP_funsearch"
shutil.rmtree(folder_path)  # 删除整个文件夹及其所有内容"""

In [ ]:
!git clone https://github.com/Perkzi/TSP_funsearch.git

import sys

sys.path.append('/content/TSP_funsearch/')
print(sys.path)

Cloning into 'TSP_funsearch'...
remote: Enumerating objects: 57, done.
remote: Counting objects: 100% (57/57), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 57 (delta 20), reused 48 (delta 11), pack-reused 0 (from 0)
Receiving objects: 100% (57/57), 100.17 KiB | 3.45 MiB/s, done.
Resolving deltas: 100% (20/20), done.
['/content', '/env/python', '/usr/lib/python311.zip', '/usr/lib/python3.11', '/usr/lib/python3.11/lib-dynload', '', '/usr/local/lib/python3.11/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.11/dist-packages/IPython/extensions', '/root/.ipython', '/content/TSP_funsearch/']


## 1. Implement LLM interface
Set the API's IP address according to your API provider (See line 65 in the following code).
```python
conn = http.client.HTTPSConnection("api.chatanywhere.com.cn")
```
You should prepare a 'key' for the LLM API. And fill them in the header (See line 76-80 in the following code).
```python
headers = {
    'Authorization': 'Bearer [put your key here, the key may start with "sk-..."]',
    'User-Agent': 'Apifox/1.0.0 (https://apifox.com)',
    'Content-Type': 'application/json'
}
```

In [ ]:
import time
import json
import multiprocessing
from typing import Collection, Any
import http.client
import traceback
from implementation import sampler



def _trim_preface_of_body(sample: str) -> str:
    """Trim the redundant descriptions/symbols/'def' declaration before the function body.
    Please see my comments in sampler.LLM (in sampler.py).
    Since the LLM used in this file is not a pure code completion LLM, this trim function is required.

    -Example sample (function & description generated by LLM):
    -------------------------------------
    This is the optimized function ...
    def priority_v2(...) -> ...:
        return ...
    This function aims to ...
    -------------------------------------
    -This function removes the description above the function's signature, and the function's signature.
    -The indent of the code is preserved.
    -Return of this function:
    -------------------------------------
        return ...
    This function aims to ...
    -------------------------------------
    去掉def之前和自己的行
    """
    lines = sample.splitlines()
    func_body_lineno = 0
    find_def_declaration = False
    for lineno, line in enumerate(lines):
        # find the first 'def' statement in the given code
        if line[:3] == 'def':
            func_body_lineno = lineno
            find_def_declaration = True
            break
    if find_def_declaration:
        code = ''
        for line in lines[func_body_lineno + 1:]:
            code += line + '\n'
        return code
    return sample

from openai import OpenAI
import time
from typing import Collection
#from TSP_algorithms.additional_prompts import base_prompt as ap  # choose a different additional_prompts
from TSP_algorithms.additional_prompts2 import step_by_step_prompts as ap

class LLMAPI:
    def __init__(self, samples_per_prompt: int, trim=True):
        API_KEY = "" # 设置 OpenAI API 密钥
        BASE_URL = 'https://api.bltcy.ai/v1'
        self.client = OpenAI(api_key=API_KEY, base_url=BASE_URL)
        self._samples_per_prompt = samples_per_prompt
        self._trim = trim
        if ap:
            self._additional_prompt = ap
        else:
            self._additional_prompt = (
                "Complete a different and more complex Python function. "
                "Be creative and you can insert multiple if-else and for-loop in the code logic. "
                "Only output the Python code, no descriptions."
            )
        if type(self._additional_prompt) is not list:
          self._additional_prompt = [self._additional_prompt]

    def draw_samples(self, prompt: str) -> Collection[str]:
        """Returns multiple predicted continuations of `prompt`."""
        #print("draw-samples")
        return [self._draw_sample(prompt) for _ in range(self._samples_per_prompt)]

    def _draw_sample(self, content: str) -> str:
        max_retries = 3

        # whether have the pre prompt before main prompt
        if len(self._additional_prompt)<=1:
          prompt = '\n'.join([content, self._additional_prompt[0]])
        elif len(self._additional_prompt)==2:
          pre_prompt = '\n'.join([content, self._additional_prompt[0]])
          retries = 0
          pre_output = ''
          prompt = ''
          while retries < max_retries:
            try:
                #print("prompt")
                response = self.client.chat.completions.create(
                    model="gpt-3.5-turbo",
                    messages=[{"role": "user", "content": pre_prompt}],
                    max_tokens=512,
                    stream=False,
                )
                print("pre_response",response)
                pre_output = response.choices[0].message.content
                # 将 第二个prompt中的 <insert optimization analysis here> 替换为分析结果。
                prompt = '\n'.join([content, self._additional_prompt[1].replace("<insert optimization analysis here>", pre_output)])
                break
                #return output
            except Exception as e:
                traceback.print_exc()
                print(f"Error occurred: {e}")
                retries += 1
                time.sleep(2)
        else:
          raise RuntimeError("list of additional_prompt should have length <= 2")

        # the main prompt
        retries = 0
        output = ''
        while retries < max_retries:
            try:
                #print("prompt")
                response = self.client.chat.completions.create(
                    model="gpt-3.5-turbo",
                    messages=[{"role": "user", "content": prompt}],
                    max_tokens=512,
                    stream=False,
                )
                print("response",response)
                output = response.choices[0].message.content

                if self._trim:
                    output = _trim_preface_of_body(output)
                break
                #return output
            except Exception as e:
                traceback.print_exc()
                print(f"Error occurred: {e}")
                retries += 1
                time.sleep(2)
        if output == '':
            print("Failed")
            raise RuntimeError("Failed after multiple retries")
        return output


llm = LLMAPI(samples_per_prompt=4)
response = llm._draw_sample('hello')
print(response)

response ChatCompletion(id='chatcmpl-BDmP5DRePbOp3vc4efFFQi2ptxAAO', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='def calculate_fibonacci(n):\n    fibonacci_series = []\n    if n <= 0:\n        return "Input must be a positive integer"\n    elif n == 1:\n        fibonacci_series.append(0)\n    elif n == 2:\n        fibonacci_series.extend([0, 1])\n    else:\n        fibonacci_series.extend([0, 1])\n        for i in range(2, n):\n            next_num = fibonacci_series[i-1] + fibonacci_series[i-2]\n            fibonacci_series.append(next_num)\n    return fibonacci_series\n\nprint(calculate_fibonacci(10))', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None))], created=1742624027, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint='fp_0165350fbb', usage=CompletionUsage(completion_tokens=121, prompt_tokens=45, total_tokens=166, completion_tokens_

## 2. Implement a 'SandBox' interface

In [ ]:
from implementation import evaluator
from implementation import evaluator_accelerate


class Sandbox(evaluator.Sandbox):
    """Sandbox for executing generated code. Implemented by RZ.

    RZ: Sandbox returns the 'score' of the program and:
    1) avoids the generated code to be harmful (accessing the internet, take up too much RAM).
    2) stops the execution of the code in time (avoid endless loop).
    """

    def __init__(self, verbose=False, numba_accelerate=True):
        """
        Args:
            verbose         : Print evaluate information.
            numba_accelerate: Use numba to accelerate the evaluation. It should be noted that not all numpy functions
                              support numba acceleration, such as np.piecewise().
        """
        self._verbose = verbose
        self._numba_accelerate = numba_accelerate

    def run(
            self,
            program: str,
            function_to_run: str,  # RZ: refers to the name of the function to run (e.g., 'evaluate')
            function_to_evolve: str,  # RZ: accelerate the code by decorating @numba.jit() on function_to_evolve.
            inputs: Any,  # refers to the dataset # self._inputs是数据集的dict，current_input是当前使用的数据集的key (名字str)
            test_input: str,  # refers to the current instance
            timeout_seconds: int,
            **kwargs  # RZ: add this
    ) -> tuple[Any, bool]:
        """Returns `function_to_run(test_input)` and whether execution succeeded.

        RZ: If the generated code (generated by LLM) is executed successfully,
        the output of this function is the score of a given program.
        RZ: PLEASE NOTE THAT this SandBox is only designed for bin-packing problem.
        返回 (function_to_run(test_input)运行结果，True)
        """
        dataset = inputs[test_input]
        try:
            # 当你有多个进程时，每个进程都会将它们的结果放入各自的 result_queue 中？
            result_queue = multiprocessing.Queue()
            # 创建一个新的进程 process，目标函数为 self._compile_and_run_function，并传递相应的参数。只并行了一个进程
            process = multiprocessing.Process(
                target=self._compile_and_run_function,
                args=(program, function_to_run, function_to_evolve, dataset, self._numba_accelerate, result_queue)
            )
            process.start()
            # 等待进程完成。如果进程在 timeout_seconds 时间内未完成，继续执行后续代码
            process.join(timeout=timeout_seconds)
            if process.is_alive():
                # 如果进程仍然在运行（超时未完成）
                # if the process is not finished in time, we consider the program illegal
                process.terminate()
                process.join()
                results = None, False
            else:
                # 如果进程已完成，从队列中获取结果。
                if not result_queue.empty():
                    results = result_queue.get_nowait()
                else:
                    results = None, False

            return results
        except:
            return None, False

    def _compile_and_run_function(self, program, function_to_run, function_to_evolve, dataset, numba_accelerate,
                                  result_queue):
        try:
            # optimize the code (decorate function_to_run with @numba.jit())
            # 加上 @numba.jit() 装饰器
            if numba_accelerate:
                program = evaluator_accelerate.add_numba_decorator(
                    program=program,
                    function_to_evolve=function_to_evolve
                )
            # compile the program, and maps the global func/var/class name to its address
            all_globals_namespace = {}
            # execute the program, map func/var/class to global namespace
            # exec 是一个内置的Python函数，用于动态执行Python代码。
            # program 是一个包含要执行代码的字符串。all_globals_namespace 是一个字典，用于保存执行代码期间创建的全局变量、函数和类
            exec(program, all_globals_namespace)
            # get the pointer of 'function_to_run'
            # 从 all_globals_namespace 中获取要运行的函数
            function_to_run = all_globals_namespace[function_to_run]
            # return the execution results
            results = function_to_run(dataset)
            # the results must be int or float
            if not isinstance(results, (int, float)):
                result_queue.put((None, False))
                return
            result_queue.put((results, True))
        except Exception:
            # if raise any exception, we assume the execution failed
            result_queue.put((None, False))

## 3. Prepare a 'specification'

In [ ]:
from TSP_algorithms import bin_packing_algorithm_0
specification = bin_packing_algorithm_0.specification
print(specification)



import numpy as np


def get_valid_bin_indices(item: float, bins: np.ndarray) -> np.ndarray:
    """Returns indices of bins in which item can fit."""
    return np.nonzero((bins - item) >= 0)[0]


def online_binpack(
        items: tuple[float, ...], bins: np.ndarray
) -> tuple[list[list[float, ...], ...], np.ndarray]:
    """Performs online binpacking of `items` into `bins`."""
    # Track which items are added to each bin.
    packing = [[] for _ in bins]
    # Add items to bins.
    for item in items:
        # Extract bins that have sufficient space to fit item.
        valid_bin_indices = get_valid_bin_indices(item, bins)
        # Score each bin based on heuristic.
        priorities = priority(item, bins[valid_bin_indices])
        # Add item to bin with highest priority.
        best_bin = valid_bin_indices[np.argmax(priorities)]
        bins[best_bin] -= item
        packing[best_bin].append(item)
    # Remove unused bins from packing.
    packing = [bin_items for bin_items i

## 4. Prepare a dataset

In [ ]:
import bin_packing_utils

bin_packing_or3 = {'OR3': bin_packing_utils.datasets['OR3']}

## 5. Start FunSearch
Please note that in jupyter notebook the following code will fail. This is because juypter does not support multiprocessing. Colab backend supports multiprocessing.

In [ ]:
from implementation import funsearch
from implementation import config
import dataclasses

# It should be noted that the if __name__ == '__main__' is required.
# Because the inner code uses multiprocess evaluation.
if __name__ == '__main__':
    class_config = config.ClassConfig(llm_class=LLMAPI, sandbox_class=Sandbox)
    config = config.Config(samples_per_prompt=4, evaluate_timeout_seconds=30)
    #config = config.Config(samples_per_prompt=1, evaluate_timeout_seconds=30,programs_database=config.ProgramsDatabaseConfig(num_islands=1))
    #global_max_sample_num = 2
    global_max_sample_num = 10  # if it is set to None, funsearch will execute an endless loop
    funsearch.main(
        specification=specification,
        inputs=bin_packing_or3,
        config=config,
        max_sample_nums=global_max_sample_num,
        class_config=class_config,
        log_dir='../logs/funsearch_llm_api'
    )


import numpy as np

 [Function(name='get_valid_bin_indices', args='item: float, bins: np.ndarray', body='    return np.nonzero((bins - item) >= 0)[0]', return_type='np.ndarray', docstring='Returns indices of bins in which item can fit.', score=None, global_sample_nums=None, sample_time=None, evaluate_time=None), Function(name='online_binpack', args='items: tuple[float, ...], bins: np.ndarray', body='    packing = [[] for _ in bins]\n    # Add items to bins.\n    for item in items:\n        # Extract bins that have sufficient space to fit item.\n        valid_bin_indices = get_valid_bin_indices(item, bins)\n        # Score each bin based on heuristic.\n        priorities = priority(item, bins[valid_bin_indices])\n        # Add item to bin with highest priority.\n        best_bin = valid_bin_indices[np.argmax(priorities)]\n        bins[best_bin] -= item\n        packing[best_bin].append(item)\n    # Remove unused bins from packing.\n    packing = [bin_items for bin_items in packing if b

INFO:absl:Best score of island 0 increased to -500.0
INFO:absl:Best score of island 1 increased to -500.0
INFO:absl:Best score of island 2 increased to -500.0
INFO:absl:Best score of island 3 increased to -500.0
INFO:absl:Best score of island 4 increased to -500.0
INFO:absl:Best score of island 5 increased to -500.0
INFO:absl:Best score of island 6 increased to -500.0
INFO:absl:Best score of island 7 increased to -500.0
INFO:absl:Best score of island 8 increased to -500.0
INFO:absl:Best score of island 9 increased to -500.0


================= Evaluated Function =================
def priority(item: float, bins: np.ndarray) -> np.ndarray:
    """Returns priority with which we want to add item to each bin.

    Args:
        item: Size of item to be added to the bin.
        bins: Array of capacities for each bin.

    Return:
        Array of same size as bins with priority score of each bin.
    """
    ratios = item / bins
    log_ratios = np.log(ratios)
    priorities = -log_ratios
    return priorities
------------------------------------------------------
Score        : -500.0
Sample time  : None
Evaluate time: 1.0512545108795166
Sample orders: None




INFO:httpx:HTTP Request: POST https://api.bltcy.ai/v1/chat/completions "HTTP/1.1 200 OK"


response ChatCompletion(id='chatcmpl-BDmTHF5LBVLcIQwOhdVQFMUbfBcQR', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='```python\ndef priority_v1(item: float, bins: np.ndarray) -> np.ndarray:\n    ratios = item / bins\n    priorities = []\n    \n    for i, ratio in enumerate(ratios):\n        if ratio < 0.2:\n            priority = 1 / (1 + np.exp(-ratio))\n        elif ratio >= 0.2 and ratio < 0.5:\n            priority = np.square(ratio)\n        else:\n            priority = np.tanh(ratio)\n        \n        priorities.append(priority)\n    \n    return np.array(priorities)\n```', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None))], created=1742624287, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint='fp_0165350fbb', usage=CompletionUsage(completion_tokens=118, prompt_tokens=185, total_tokens=303, completion_tokens_details=None, prompt_tokens

...

response ChatCompletion(id='chatcmpl-BDmTSWVJeqhxcHuZG5UN9EMlOIOOG', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='def priority_v2(item: float, bins: np.ndarray) -> np.ndarray:\n    """Even more improved version of priorities."""\n    ratios = item / bins\n    priorities = np.zeros(len(bins))\n    \n    for i in range(len(bins)):\n        if ratios[i] > 0.7:\n            priorities[i] = 3 * ratios[i]\n        elif ratios[i] > 0.5:\n            priorities[i] = 2.5 * ratios[i]\n        elif ratios[i] > 0.3:\n            priorities[i] = 2 * ratios[i]\n        else:\n            priorities[i] = ratios[i] * np.log(ratios[i])\n    \n    for i in range(len(bins)):\n        if priorities[i] < 1:\n            priorities[i] = 1 + np.exp(priorities[i])\n            \n    return priorities', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None))], created=1742624298, model='gpt-3.5-turbo-0125', obje

INFO:httpx:HTTP Request: POST https://api.bltcy.ai/v1/chat/completions "HTTP/1.1 200 OK"


response ChatCompletion(id='chatcmpl-BDmTWfR0DBjM5AWzmseG24Xr5yMLL', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='def priority_v2(item: float, bins: np.ndarray) -> np.ndarray:\n    ratios = item / bins\n    priorities = np.zeros(len(bins))\n    \n    for i in range(len(bins)):\n        if ratios[i] > 0.7:\n            priorities[i] = 3 * ratios[i]\n        elif ratios[i] > 0.5:\n            priorities[i] = 2 * ratios[i]\n        elif ratios[i] > 0.3:\n            priorities[i] = 1.5 * ratios[i]\n        else:\n            priorities[i] = ratios[i] * np.log(ratios[i])\n        \n        if bins[i] < 10:\n            priorities[i] *= 1.2\n        \n    return priorities', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None))], created=1742624302, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint='fp_0165350fbb', usage=CompletionUsage(completion_t

INFO:httpx:HTTP Request: POST https://api.bltcy.ai/v1/chat/completions "HTTP/1.1 200 OK"


response ChatCompletion(id='chatcmpl-BDmTXwdzIVvq99g7eqf4Mjgfm9AVd', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='def priority_v2(item: float, bins: np.ndarray) -> np.ndarray:\n    ratios = item / bins\n    priorities = np.zeros(len(bins))\n    \n    for i in range(len(bins)):\n        if ratios[i] > 0.7:\n            priorities[i] = 2.5 * ratios[i]\n        elif ratios[i] > 0.5:\n            priorities[i] = 2 * ratios[i]\n        elif ratios[i] > 0.3:\n            priorities[i] = 1.5 * ratios[i]\n        elif ratios[i] > 0.1:\n            priorities[i] = ratios[i] * np.log(ratios[i])\n        else:\n            priorities[i] = 0.5 * ratios[i] + np.sqrt(ratios[i])\n    \n    return priorities', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None))], created=1742624303, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint='fp_0165350fbb', usage=Com